In [1]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from prophet import Prophet

plt.rcParams['figure.figsize'] = (20,10)
plt.style.use('fivethirtyeight')

SyntaxError: ignored

In [5]:
def extract_geographical_revenue(symbol):
    api_key = '680c64465fdbbc5b65b3fe116ac89d18'
    url = f'https://financialmodelingprep.com/api/v4/revenue-geographic-segmentation?symbol={symbol}&period=quarter&structure=flat&apikey={api_key}'
    raw_df = requests.get(url).json()


    return raw_df



In [7]:
df_geographical = extract_geographical_revenue('META')
df_geographical


{'Error Message': 'Special Endpoint : This endpoint is not available under your current subscription please visit our subscription page to upgrade your plan at https://site.financialmodelingprep.com/developer/docs/pricing'}

In [ ]:
# Convert JSON to DataFrame
    data = []
    for entry in raw_df:
        for date, values in entry.items():
            row = {'Date': date}
            for country, value in values.items():
                row[country] = value
            data.append(row)

    df = pd.DataFrame(data)


In [4]:
raw_df

NameError: ignored

In [3]:
df_geographical = extract_geographical_revenue('META')
df_geographical.head()

AttributeError: ignored

In [ ]:
data

NameError: ignored

In [ ]:
df

NameError: ignored

In [ ]:
df_geographical = extract_geographical_revenue('TSLA')
df_geographical.head()

AttributeError: ignored

In [9]:
def extract_product_revenue(symbol):
    api_key = '680c64465fdbbc5b65b3fe116ac89d18'
    url = f'https://financialmodelingprep.com/api/v4/revenue-product-segmentation?symbol={symbol}&period=quarter&structure=flat&apikey={api_key}'
    raw_df = requests.get(url).json()
    # Create an empty DataFrame
    df = pd.DataFrame()

    # Iterate through the data and append to the DataFrame
    for entry in raw_df:
        for date, values in entry.items():
            df = pd.concat([df, pd.DataFrame.from_dict({date: values}, orient='index')])

    # Reset the index
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'Date'}, inplace=True)

    return df

df_product = extract_product_revenue('TSLA')
df_product.head()

AttributeError: ignored

In [ ]:
# Changing all the NaN values to zero for further modifications.
df_geographical = df_geographical.fillna(0)

# Merging all the United States columns.
df_geographical['United States'] = df_geographical['UNITED STATES'] + df_geographical['U S'] + df_geographical['U']

# Merging all the China colums.
df_geographical['China'] = df_geographical['CHINA'] + df_geographical['C N']

# Drop the unused columns.
df_geographical = df_geographical.drop(["UNITED STATES" ,"U S", 'U','CHINA', 'C N'],axis =1)

# Reversing the rows to make it from earlier to newer.
df_geographical = df_geographical .iloc[::-1]

# Reseting the index
df_geographical = df_geographical.reset_index().drop(['index'],axis=1)
df_geographical.head()

# Set the 'DATE' column as the index
df_geographical = df_geographical.set_index('Date')
df_geographical.index = pd.to_datetime(df_geographical.index)

# Dropping the NA values.
df_geographical.dropna(inplace = True)

df_geographical.head()

In [10]:
# Visualize the time series to find stationarity and patterns

plt.plot(df_geographical.index, df_geographical['United States'])
plt.xlabel('Quarters')
plt.ylabel('United States production')
plt.title('Production from United States each quarter')
plt.show()

AttributeError: ignored

Production across the United States of Tesla is increasing with increasing slope quarter by quarter.

The ADF test helps determine if a time series is stationary or not. It provides the ADF statistic, p-value, and critical values for different significance levels,

If the p-value is less than a chosen significance level (e.g., 0.05), we can reject the null hypothesis and conclude that the time series is stationary. Otherwise, if the p-value is greater than the significance level, we fail to reject the null hypothesis, suggesting that the time series is non-stationary.

In [ ]:
# Perform the Augmented Dickey-Fuller test

result = adfuller(df_geographical['United States'])
print('ADF Statistic:', result[0])
print('p-value:', result[1])
print('Critical Values:')
for key, value in result[4].items():
    print(key, ':', value)

Trend Analysis
Trend helps to uncover the underlying direction or pattern in the time series, which can be valuable for forecasting and decision-making.

To perform trend analysis on a time series, we can use various techniques, including visual inspection, moving averages, and regression analysis.

Visual Inspection: Plotting the time series data can often reveal the presence of a trend. A clear upward or downward movement over time suggests the presence of a trend component. Visual inspection allows you to observe the overall pattern and identify any deviations or changes in the series.

Moving Averages: Moving averages are widely used for trend analysis. They help smooth out short-term fluctuations in the data, making it easier to identify the underlying trend. Common types of moving averages include the simple moving average (SMA), weighted moving average (WMA), and exponential moving average (EMA).

Here are the three common types of moving averages used for trend analysis:

Simple Moving Average (SMA): The Simple Moving Average calculates the average of a specified number of data points over a defined window. Each data point contributes equally to the average, and older observations are equally weighted as newer ones. The SMA provides a smoothed representation of the data by reducing random fluctuations.

Weighted Moving Average (WMA): The Weighted Moving Average assigns different weights to the data points within the window. The weights can be linear or follow a specific pattern. The WMA gives more emphasis to recent observations, allowing it to respond more quickly to changes in the data compared to the SMA.

Exponential Moving Average (EMA): The Exponential Moving Average is a type of weighted moving average that assigns exponentially decreasing weights to the data points. It places more weight on recent observations while gradually reducing the importance of older observations. The EMA is more responsive to recent changes in the data and is often used in technical analysis.

Regression Analysis: Regression analysis can be applied to estimate and visualize the trend component of a time series. It involves fitting a regression model to the data, where time is considered as an independent variable and the variable of interest is the dependent variable. The trend component can then be extracted from the regression model.

In [ ]:
# Calculate the rolling mean (simple moving average) with a window size of 3 months
rolling_mean = df_geographical['United States'].rolling(window=3).mean()

# Visualize the original time series and the trend component
plt.plot(df_geographical.index, df_geographical['United States'], label='Original')
plt.plot(df_geographical.index, rolling_mean, color='red', label='Trend (Moving Average)')
plt.xlabel('Quarter')
plt.ylabel('Production from United States')
plt.title('Trend Analysis: United States Production')
plt.legend()
plt.show()

Training the model and forecasting
Once the data is in proper format we are now ready to train the model and predict the future. Here we go:

In [ ]:
# Format data for prophet model using ds and y
df_geographical = df_geographical.reset_index() \
    .rename(columns={'Date':'ds',
                     'United States':'y'})

# Setup and train model and fit
model = Prophet()
model.fit(df_geographical)

# define the period for which we want a prediction
future = ['2023-09-30','2023-12-30','2024-03-30','2024-06-30','2024-09-30','2024-12-30','2025-03-30','2025-06-30','2025-09-30']
future = pd.DataFrame(future)
future.columns = ['ds']
future['ds']= pd.to_datetime(future['ds'])

forecast = model.predict(future)

# plot forecast
model.plot(forecast)
plt.show()